In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2 

In [2]:
import pywren
import time
from tensorflow.examples.tutorials.mnist import input_data
# %aimport linear_models
from linear_models import LinearOneVsAllClassifier
import numpy as np
from helper import generate_data
from itertools import product
from cvxopt import matrix, solvers

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist_images = np.copy(mnist.train.images)
mnist_labels = np.argmax(mnist.train.labels, axis=1)
mnist_test_images = mnist.test.images
mnist_test_labels = np.argmax(mnist.test.labels, axis=1)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
models = [LinearOneVsAllClassifier(10, np.load('test/w_{}.npy'.format(i)), np.load('test/b_{}.npy'.format(i)))
          for i in range(5)]

In [5]:
X_exp, Y_exp = generate_data(5, mnist_images, mnist_labels, models)

In [6]:
[model.evaluate(X_exp, Y_exp) for model in models]

[1.0, 1.0, 1.0, 1.0, 1.0]

In [7]:
import scipy.optimize as opt

def tryRegionOneVsAllScipy(models, labels, x, delta=1e-6, attempts=0):
    
    def objective(v):
        return .5  * v.dot(v)

    def gradient(v):
        return v
    
    h = []
    G = []
    num_models = len(models)
    for i in xrange(num_models):
        others = range(10)
        target = labels[i]
        del others[target]
        target_w, target_b = models[i].weights[target], models[i].bias[target]
        for j in others:
            other_w, other_b = models[i].weights[j], models[i].bias[j]
            ineq_val = np.dot(target_w - other_w, x) + target_b - other_b - delta
            h.append(ineq_val)
            G.append(other_w - target_w)

    def cons_i(z, i):
        return -1 * G[i].dot(z) + h[i]
    
    cons_per_i = [{'type':'ineq', 'fun': lambda z, i=i: cons_i(z, i)} for i in np.arange(len(G))]

    sol = opt.minimize(objective, np.zeros(x.shape[0]), jac=gradient, constraints=cons_per_i, 
                       options={'disp': False}, method='SLSQP')
    if sol.success:
        v = sol.x
        perturbed_x = np.array(x + v).reshape(1, -1)
        is_desired_label = [models[i].predict(perturbed_x)[0] == labels[i] for i in xrange(num_models)]
        if sum(is_desired_label) == num_models:
            return v
        else:
            attempts += 1
            return tryRegionOneVsAllScipy(models, labels, x, delta * 1.5, attempts)
    else:
        return None

In [23]:
import ray
import time

# ray.init()

@ray.remote
def function(x):
    time.sleep(10)
    return x

arglist = [1, 2, 3, 4]
s = time.time()
result_ids = [function.remote(x) for x in arglist]
resultlist = ray.get(result_ids)
print time.time() - s 
print "done"

10.013655901
done


In [8]:
def tryRegionOneVsAll(models, labels, x, delta=1e-8):
    P = matrix(np.identity(x.shape[0]))
    q = matrix(np.zeros(x.shape[0]))
    h = []
    G = []
    num_models = len(models)
    for i in xrange(num_models):
        others = range(10)
        target = labels[i]
        del others[target]
        target_w, target_b = models[i].weights[target], models[i].bias[target]
        for j in others:
            other_w, other_b = models[i].weights[j], models[i].bias[j]
            ineq_val = np.dot(target_w - other_w, x) + target_b - other_b - delta
            h.append(ineq_val)
            G.append(other_w - target_w)
    h = matrix(h)
    G = matrix(np.array(G))
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h)
    if sol['status'] == 'optimal':
        v = np.array(sol['x']).reshape(-1,)
        perturbed_x = np.array(x + v).reshape(1, -1)
        is_desired_label = [models[i].predict(perturbed_x)[0] == labels[i] for i in xrange(num_models)]
        if sum(is_desired_label) == num_models:
            return v
        else:
            return tryRegionOneVsAll(models, labels, x, delta * 1.5)
    else:
        return None

In [9]:
import quadprog

In [10]:
def tryRegionOneVsAllQuadProg(models, labels, x, delta=1e-10):
    P = np.identity(x.shape[0])
    q = np.zeros(x.shape[0])
    h = []
    G = []
    num_models = len(models)
    for i in xrange(num_models):
        others = range(10)
        target = labels[i]
        del others[target]
        target_w, target_b = models[i].weights[target], models[i].bias[target]
        for j in others:
            other_w, other_b = models[i].weights[j], models[i].bias[j]
            ineq_val = np.dot(target_w - other_w, x) + target_b - other_b - delta
            h.append(ineq_val)
            G.append(other_w - target_w)
    h =  -1 * np.array(h)
    G =  -1 * np.array(G).T
    sol = quadprog.solve_qp(P, q, G, h)
    return sol[0]
#     if sol['status'] == 'optimal':
#         v = np.array(sol['x']).reshape(-1,)
#         perturbed_x = np.array(x + v).reshape(1, -1)
#         is_desired_label = [models[i].predict(perturbed_x)[0] == labels[i] for i in xrange(num_models)]
#         if sum(is_desired_label) == num_models:
#             return v
#         else:
#             return tryRegionOneVsAll(models, labels, x, delta * 1.5)
#     else:
#         return None

In [ ]:
v = tryRegionOneVsAllQuadProg(models, [4,2,3,3,5], X_exp[0])

In [ ]:
%timeit sol = tryRegionOneVsAllScipy(models, [4,2,3,3,5], X_exp[0])

In [ ]:
%timeit sol = tryRegionOneVsAll(models, [4,2,3,3,5], X_exp[0])

In [ ]:
%timeit sol = tryRegionOneVsAllQuadProg(models, [4,2,3,3,5], X_exp[0])

In [ ]:
%lprun -f  tryRegionOneVsAll tryRegionOneVsAll(models, [1,2,3,4,5], X_exp[0])

In [ ]:
for i in range(25):
    labels = np.random.choice(10, 5)
    sol = tryRegionOneVsAllQuadProg(models, labels, X_exp[0])
    sol2= tryRegionOneVsAll(models, labels, X_exp[0])
    print np.linalg.norm(sol), np.linalg.norm(sol2)
    print labels
    preds = np.array([model.predict((X_exp[0] + sol).reshape(1,-1))[0] for model in models])
    print preds
    print (preds == labels).sum()

In [ ]:
all_regions = product(range(10), repeat=len(models))
size = 50
main = 10 ** len(models) / size
remaining = 10 ** len(models) - main * size 
sub_lists = [[next(all_regions) for _ in xrange(size)] for _ in xrange(main)]
sub_lists.append([next(all_regions) for _ in xrange(remaining)])

In [ ]:
sum([len(x) for x in sub_lists])

In [ ]:
len(sub_lists)

In [11]:
all_regions = product(range(10), repeat=len(models))
sub_lists = [[next(all_regions) for _ in xrange(1)] for _ in xrange(3)]

In [12]:
sum([len(x) for x in sub_lists])

480

In [13]:
from multiprocessing import Pool

In [18]:
len(sub_lists)

3

In [24]:
def stop(x):
    time.sleep(10)
    return x

start = time.time()
p = Pool(4)
result = p.map(stop, [1,2,3,4])
print time.time() - start

10.0278580189


In [15]:
def test(labels_list):
    return [tryRegionOneVsAll(models, labels, X_exp[0]) for labels in labels_list]

In [ ]:
`
start = time.time()
p = Pool(3)
result = p.map(test, sub_lists)
print time.time() - start 

In [ ]:
start = time.time()
def test(labels_list):
    return [tryRegionOneVsAllScipy(models, labels, X_exp[0]) for labels in labels_list]
pwex = pywren.default_executor()
a = time.time()
futures = pwex.map(test, sub_lists)
print "TIME", time.time() - a
results = pywren.get_all_results(futures)
print "Total Time", start - time.time()

In [ ]:
sum([len(res) for res in results])

In [ ]:
!pywren print_latest_logs

In [ ]:
len(results_with_pywren_map)